# Carregando variáveis de ambiente

In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")

pinecone_api_ley = config["PINECONE_API_KEY"]
pinecone_env = config["PINECONE_ENVIRONMENT"]
index_pinecone = config["INDEX_NAME"]

openai_api_key = config["OPENAI_API_KEY"]

# Verificando conexão com o pinecone

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_ley)
index = pc.Index(index_pinecone)

print(pc.list_indexes())

c:\Users\irani\Desktop\Insper Metricis\Plataforma Metricis\Dados\Extracao-base\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{
    "name": "livreto-base-evidencia",
    "metric": "cosine",
    "host": "livreto-base-evidencia-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": {
        "embedding_model": "text-embedding-3-small"
    }
}, {
    "name": "ods-onu",
    "metric": "cosine",
    "host": "ods-onu-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 512,
    "deletion_protection": "disabled",
    "tags": {
        "embedding_model": "text-embedding-3-small"
    }
}]


# Extraindo texto do pdf (ods_onu.pd) e salvando em formato json (ods_onu.json)

In [ ]:
import pdfplumber
import json

path = "./data/ods_onu.pdf"
pages=[]
with pdfplumber.open(path) as pdf:
    for i , p in enumerate(pdf.pages, start=1):
        if i < 18 or i > 39:
            continue
        text = p.extract_text() or ""
        pages.append({"page": i, "text": text})

with open("json/ods_onu.json","w",encoding="utf-8") as f:
    json.dump(pages,f,ensure_ascii=False,indent=2)

# Limpando textos extraídos no json e salvando novo json limpo

In [ ]:
import re 

def limpar_texto_ods(texto: str) -> str:
    if not texto:
        return ""

    # Mantemos sua lógica, mas garantimos que pontos entre números (1.1) não sumam
    texto = re.sub(r'[^\wÀ-ÿ\s.,;:!?()\-"“”]', ' ', texto)
    
    # Remove letras soltas (OCR)
    texto = re.sub(r'(?:\b[A-ZÀ-Ý]\b\s*){3,}', ' ', texto)

    # Normalização de quebras e espaços
    texto = re.sub(r'\n{2,}', '\n', texto)
    texto = re.sub(r'\n', ' ', texto)
    texto = re.sub(r'\s{2,}', ' ', texto)

    return texto.strip()


def limpar_json_ods(lista_ods: list) -> list:
    json_limpo = []
    for pagina in lista_ods:
        # Aqui, 'text' deve conter a descrição da ODS e suas Metas
        texto_limpo = limpar_texto_ods(pagina.get("text", ""))
        
        if texto_limpo:
            json_limpo.append({
                "page": pagina["page"],
                "text": texto_limpo
            })
    return json_limpo



with open('json/ods_onu.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)


ods_limpas = limpar_json_ods(raw_data)

with open("json/ods_onu_clean.json", "w", encoding="utf-8") as f:
    json.dump(ods_limpas, f, ensure_ascii=False, indent=2)

# Fazendo chunks

In [24]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


with open('json/ods_onu_clean.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def preparar_documentos_ods(data):
    docs_com_contexto = []
    
    for item in data:
        texto = item['text']
        # Divide o texto em sentenças ou metas baseando-se no padrão "Objetivo X" ou "X.X"
        # Aqui quebramos onde começa um novo Objetivo ou uma nova Meta
        partes = re.split(r'(Objetivo \d+\.)', texto)
        
        for i in range(1, len(partes), 2):
            titulo_ods = partes[i].strip()  # Ex: "Objetivo 1."
            conteudo_ods = partes[i+1].strip() # O texto que vem depois
            
            # Se for o detalhamento de metas (ex: 1.1, 1.2)
            # Vamos garantir que o título da ODS acompanhe cada bloco
            doc = Document(
                page_content=f"{titulo_ods} {conteudo_ods}",
                metadata={
                    "page": item['page'],
                    "ods": titulo_ods
                }
            )
            docs_com_contexto.append(doc)
            
    return docs_com_contexto

documents = preparar_documentos_ods(data)
documents

[Document(metadata={'page': 18, 'ods': 'Objetivo 1.'}, page_content='Objetivo 1. Acabar com a pobreza em todas as suas formas, em todos os lugares'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 2.'}, page_content='Objetivo 2. Acabar com a fome, alcançar a segurança alimentar e melhoria da nutrição e promover a agricultura sustentável'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 3.'}, page_content='Objetivo 3. Assegurar uma vida saudável e promover o bem-estar para todos, em todas as idades'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 4.'}, page_content='Objetivo 4. Assegurar a educação inclusiva e equitativa e de qualidade, e promover oportunidades de aprendizagem ao longo da vida para todos'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 5.'}, page_content='Objetivo 5. Alcançar a igualdade de gênero e empoderar todas as mulheres e meninas'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 6.'}, page_content='Objetivo 6. Assegurar a disponibilidade e gestão sust